In [ ]:
# !pip install requests
# !pip install pillow

In [5]:
# Note: DALL-E 3 requires version 1.0.0 of the openai-python library or later
import os
from openai import AzureOpenAI
import json
import requests
from PIL import Image
import html  
import re
from dotenv import load_dotenv

load_dotenv() 

if '<redacted' in os.getenv("openai_dalle3_api_base") or os.getenv("openai_dalle3_api_base") == None:
    print ('Update .env with your Azure OpenAI Service details. Refer to sample.env for example.')

In [ ]:
def remove_unicode_characters(input_string):  
    # This regex pattern matches any character that is not in the range of ASCII  
    ascii_only = re.sub(r'[^\x00-\x7F]+', '', input_string)  
    return ascii_only  

In [6]:
# Set the directory for the stored image
image_dir = 'images'

# If the directory doesn't exist, create it
if not os.path.isdir(image_dir):
    os.mkdir(image_dir)


In [7]:
client = AzureOpenAI(
    api_version=os.getenv('openai_dalle3_api_version'),
    azure_endpoint=os.getenv('openai_dalle3_api_base'),
    api_key=os.getenv('openai_dalle3_api_key')
)


In [9]:
print ('Loading products...')
products = []
with open("products.jsonl") as f_in:
    for line in f_in:
        products.append(line)

total_products = len(products)
print ('Total Products:', total_products)


Loading products...
Total Products: 18


In [17]:
# Create the images
counter = 0
for line in products:
    try:
        json_data = json.loads(line)
        picture_file_name = json_data['Title'] + '.png'
        print ('Creating image for:', json_data['Title'])
        if os.path.exists( os.path.join(image_dir, picture_file_name)) == False:
            result = client.images.generate(
                model="dall-e-3",
                prompt=remove_unicode_characters(json_data['Description']),
                n=1
            )
            
            image_url = json.loads(result.model_dump_json())['data'][0]['url']
    
            # Initialize the image path (note the filetype should be png)
            image_path = os.path.join(image_dir, picture_file_name)
            
            # Retrieve the generated image
            generated_image = requests.get(image_url).content  # download the image
            with open(image_path, "wb") as image_file:
                image_file.write(generated_image)
            
            # Display the image in the default image viewer
            # image = Image.open(image_path)
            # image.show()
            # break
    except Exception as ex:
        print ('Error:', ex)
    
    counter += 1
    if counter%100== 0:
        print ('Compled:', counter, 'of', total_products)
    
print ('Completed:', counter, 'of', total_products)

    

Creating image for: ContosoMax Pro 5G Smartphone
Creating image for: ContosoLite X1 Dual-SIM Mobile
Creating image for: ContosoNova Ultra HD Camera Phone
Creating image for: Contoso Retail UltraBook Pro X150
Creating image for: Contoso Retail Gamer's Choice G730
Creating image for: Contoso Retail Business Elite B500
Creating image for: Contoso Retail Frost-Free Double Door Refrigerator - 350L
Creating image for: Contoso Retail Smart Inverter Side-by-Side Refrigerator - 500L
Creating image for: Contoso Retail Energy Efficient Bottom Freezer Refrigerator - 400L
Creating image for: Contoso Retail UltraClean Front Load Washing Machine
Creating image for: Contoso Retail EcoWash Top Load Washing Machine
Creating image for: Contoso Retail SmartWash WiFi-Enabled Washing Machine
Creating image for: The Enchanted Forest: A Contoso Retail Adventure
Creating image for: Mystery at Midnight: A Contoso Retail Bestseller
Creating image for: The Lost Kingdom Chronicles: A Contoso Retail Fantasy Epic
Er